# How to use the chat-gpt-jupyter-extension

This is a browser extension that brings ChatGPT **into** your Jupyter
notebooks. The way I always describe Jupyter to my friends is that *it's a
tool that handles the mundane task of writing things down for you*. When I saw
ChatGPT, I thought that there was no better home for ChatGPT than inside my
Jupyter notebooks.

This extension will reuse an existing login session that you must first
establish with [OpenAI's ChatGPT service](https://chat.openai.com/). If you
aren't signed in already, or your session timed out, it will prompt you to
sign in. 

It only works with the classic Jupyter notebook user interface. Help is
welcome to make it work with JupyterLab. It works with both local Jupyter
notebooks that you access via http://localhost or notebooks hosted on the
public MyBinder service at https://mybinder.org. 

If you want it to work with Jupyter notebooks from a different URL, you'll
need to edit the `manifest.json` file. It works by injecting script from the
extension into the Jupyter notebook web page and asking the extension to use
the bearer token obtained by logging into the Open AI service to communicate
with the service; the browser extension effectively acts as a privileged
proxy. 

This is the only reliable way that I've found to do this as OpenAI hasn't
released an official API for the ChatGPT service. 


## Prompt templates

In this section, I'm going to include a few templates that I've been experimenting with.
The way you can think about templates is that the extension will automatically prepend
your chat query with the referenced prompt. 

### Hybrid chat session
This is the main reason why I created this extension: to give ChatGPT access to a Jupyter
kernel. ChatGPT is pretty bad at anything that requires precision and/or accuracy. The
canonical example of this is arithmetic.

In the following example, there are two cells. The first cell is the prompt *template*:

```
##### prompt:code
In your reply, only return Python code. The Python code must be in a function. Don't describe what the function does. Don't provide comments in the function. Make sure that you write the call to the function, and print out the result of that call. In the print statement, provide a brief description of the result and the units of the result, if known.

Write a Python program that:
```

The second cell contains the user prompt which also specifies which template to use: `code`.

```
##### chat
/prompt:code

What is 323239857 * 23492?
```

The content of the `code` template will appear before the user query `What is 323239857 * 23492?` in what is sent to ChatGPT.

##### prompt:code

In your reply, only return Python code. The Python code must be in a function. Don't describe what the function does. Don't provide comments in the function. Make sure that you write the call to the function, and print out the result of that call. In the print statement, provide a brief description of the result and the units of the result, if known.

Write a Python program that:


##### chat
/prompt:code
/nochain

What is 323239857 * 23492?

##### response generated  by ChatGPT in 4 seconds at 1/7/2023 5:34:13 PM
```python
def multiply():
    return 323239857 * 23492
print(multiply())  # 758299835894
```

In [39]:
def multiply():
    return 323239857 * 23492
print(multiply())  # 758299835894

7593550720644


##### chat
/nochain
What is 323239857 * 23492?

##### response generated  by ChatGPT in 2 seconds at 1/7/2023 5:34:43 PM

323239857 * 23492 = 7562688553924

## Calculations involving memorized facts
The following examples require ChatGPT to lookup some memorized facts. All of the examples
here are simple equations that can be looked up online. For some of these, it's always 
going to remember the right equation to use. For others, it's hit or miss with varying rates
of success. 

The examples that I'm saving in the notebook will show a certain set of results to illustrate 
the point that I'm making. If you rerun those examples, you may get different results.

The three responses to the question below were run at different times and produced different results. I wanted to use a width of 24 units and a distance of 28 units. You can see in the recalled equation that ChatGPT:

- Uses two different methods to convert radians to degrees
- Divides width by 2 in one case, and not in the other case

Only in the final case did it generate a result that used the correct form of the equation.

For reference, the correct equation[1] is:

$visualAngle=2*atan(\frac{(width/2)}{distance})*\frac{180}{\pi}$

[1] https://elvers.us/perception/visualAngle/va.html

##### chat
/prompt:code 
/nochain

Calculate the number of horizontal degrees that an object occupies in your field of view if you know the width of the object and its distance from you. Make sure you return the width in degrees. 

##### response generated  by ChatGPT in 8 seconds at 1/7/2023 5:39:34 PM
```python
def horizontal_degrees(width, distance):
    import math
    return math.degrees(math.atan(width / distance))
print(horizontal_degrees(10, 50))  # Width of 10 at a distance of 50 is 14.0 degrees
```

In [47]:
def horizontal_degrees(width, distance):
    import math
    return math.degrees(math.atan(width / distance))
print(horizontal_degrees(24, 28)) 

40.60129464500447


##### response generated  by ChatGPT in 6 seconds at 1/7/2023 5:35:07 PM
```python
def horizontal_degrees(width, distance):
    import math
    return math.atan2(width / 2, distance) * 180 / math.pi
print(horizontal_degrees(5, 10), "degrees")
```

In [40]:
def horizontal_degrees(width, distance):
    import math
    return math.atan2(width / 2, distance) * 180 / math.pi
print(horizontal_degrees(24, 28), "degrees")

23.19859051364819 degrees


##### response generated  by ChatGPT in 7 seconds at 1/7/2023 4:51:04 PM
```python
def horizontal_degrees(width, distance):
    return (2 * math.atan2(width / 2, distance)) * (180 / math.pi)
print(horizontal_degrees(4, 100), "degrees")
```

In [35]:
def horizontal_degrees(width, distance):
    return (2 * math.atan2(width / 2, distance)) * (180 / math.pi)
print(horizontal_degrees(24, 28), "degrees")

46.39718102729638 degrees


## A simpler equation: volume of a sphere

In this case, I didn't notice ChatGPT getting this wrong. I suspect the reason for this is
because there is no ambiguity in the volume of a sphere equation. I suspect it was trying
to get fancy with the visual angle equation by trying to prematurely simplify it without 
a good understanding of the rules of algebraic simplification, or that it was trained on
a number of examples where incorrect simplifications were present.

##### chat
/prompt:code
/nochain

Calculate the volume of the planet earth.

##### response generated  by ChatGPT in 13 seconds at 1/7/2023 5:56:43 PM
```python
def earth_volume():
    radius = 6371  # in kilometers
    volume = 4/3 * 3.14 * radius**3
    return volume
print("Earth volume:", earth_volume(), "km^3")
```

In [48]:
def earth_volume():
    radius = 6371  # in kilometers
    volume = 4/3 * 3.14 * radius**3
    return volume
print("Earth volume:", earth_volume(), "km^3")

Earth volume: 1082657777102.0533 km^3


##### response generated  by ChatGPT in 8 seconds at 1/7/2023 4:52:29 PM
```python
def earth_volume():
  diameter = 12_742_000
  radius = diameter / 2
  volume = 4/3 * 3.14 * radius**3
  return volume
print(f"The volume of Earth is {earth_volume()} cubic meters")
```

In [36]:
def earth_volume():
  diameter = 12_742_000
  radius = diameter / 2
  volume = 4/3 * 3.14 * radius**3
  return volume
print(f"The volume of Earth is {earth_volume()} cubic meters")

The volume of Earth is 1.0826577771020533e+21 cubic meters


##### chat
/prompt:code
/nochain

On his day off, Lee took a trip to the zoo. He drove an average speed of 65mph and it took him 2.5 hours to get from his house to the zoo. How far is the zoo from his house?

##### response generated  by ChatGPT in 5 seconds at 1/7/2023 4:53:08 PM
```python
def distance_to_zoo(speed, time):
    return speed * time
print(distance_to_zoo(65, 2.5), "miles")
```

In [37]:
def distance_to_zoo(speed, time):
    return speed * time
print(distance_to_zoo(65, 2.5), "miles")

162.5 miles


##### chat
/prompt:code
/nochain

Bill took a trip to see a friend. His friend lives 225 miles away. He drove in town at an average of 30 mph, then he drove on the interstate at an average of 70 mph. The trip took three-and-a-half hours total. How far did Bill drive on the interstate?

##### response generated  by ChatGPT in 9 seconds at 1/7/2023 3:07:55 PM
```python
def distance_driven(total_distance, total_time, town_speed, interstate_speed):
  town_time = total_time / 2
  town_distance = town_time * town_speed
  interstate_distance = total_distance - town_distance
  return interstate_distance
print(distance_driven(225, 3.5, 30, 70))
```

In [23]:
def distance_driven(total_distance, total_time, town_speed, interstate_speed):
  town_time = total_time / 2
  town_distance = town_time * town_speed
  interstate_distance = total_distance - town_distance
  return interstate_distance
print(distance_driven(225, 3.5, 30, 70))

172.5


##### prompt:chemistry

In your reply, only return Python code. Make sure you define all constants and their units in the Python program. Do not return any explanatory text, return only the program. If you generate a function, make sure you write the call to that function as well and print out the result of calling that function to the console and some text explaining what the value that you printed is. Write a Python program that:

##### chat
/prompt:chemistry
/nochain

Determine the volume, in liters, occupied by 2.34 grams of carbon dioxide gas at STP.

##### response generated  by ChatGPT in 15 seconds at 1/7/2023 3:26:06 PM
```python
def volume_of_gas_at_stp(mass: float, molar_mass: float, temperature: float, pressure: float) -> float:
    R = 8.31 # J/mol*K
    return (mass * R * temperature) / (pressure * molar_mass)
print(volume_of_gas_at_stp(mass=2.34, molar_mass=44.01, temperature=273.15, pressure=101325))
# Output: 0.004438113615333737 liters
```

In [30]:
def volume_of_gas_at_stp(mass: float, molar_mass: float, temperature: float, pressure: float) -> float:
    R = 8.31 # J/mol*K
    return (mass * R * temperature) / (pressure * molar_mass)
print(volume_of_gas_at_stp(mass=2.34, molar_mass=44.01, temperature=273.15, pressure=101325))
# Output: 0.004438113615333737 liters

0.0011911051572795427


##### response generated  by ChatGPT in 18 seconds at 1/7/2023 3:19:00 PM
```python
AVOGADRO_CONSTANT = 6.02214076 * 10**23
MOLAR_MASS_CO2 = 44.01
def volume_of_gas(mass, molar_mass, pressure, temperature):
  volume = mass / (molar_mass * pressure * temperature)
  return volume
pressure = 101.325 # kPa
temperature = 273.15 # K
volume = volume_of_gas(2.34, MOLAR_MASS_CO2, pressure, temperature)
volume_in_liters = volume / 1000
print(volume_in_liters)
```

In [28]:
AVOGADRO_CONSTANT = 6.02214076 * 10**23
MOLAR_MASS_CO2 = 44.01
def volume_of_gas(mass, molar_mass, pressure, temperature):
  volume = mass / (molar_mass * pressure * temperature)
  return volume
pressure = 101.325 # kPa
temperature = 273.15 # K
volume = volume_of_gas(2.34, MOLAR_MASS_CO2, pressure, temperature)
volume_in_liters = volume / 1000
print(volume_in_liters)

1.921085400660875e-09


##### response generated  by ChatGPT in 7 seconds at 1/4/2023 8:20:33 PM
```python
def volume_of_gas(grams, gas):
    molar_mass = 44.01 if gas == 'CO2' else 0
    return (grams / molar_mass) * 22.4
print(volume_of_gas(2.34, 'CO2'))
```

In [1]:
def volume_of_gas(grams, gas):
    molar_mass = 44.01 if gas == 'CO2' else 0
    return (grams / molar_mass) * 22.4
print(volume_of_gas(2.34, 'CO2'))

1.191002044989775


##### response generated  by ChatGPT in 18 seconds at 1/4/2023 8:15:36 PM
```python
def volume_of_co2_at_stp(mass):
    MOLAR_MASS_OF_CO2 = 44.01 # g/mol
    NUMBER_OF_MOLECULES_IN_A_MOLE = 6.02214076e23 # molecules/mol
    VOLUME_AT_STP = 22.4 # L/mol
    moles = mass / MOLAR_MASS_OF_CO2
    molecules = moles * NUMBER_OF_MOLECULES_IN_A_MOLE
    volume = molecules * VOLUME_AT_STP / NUMBER_OF_MOLECULES_IN_A_MOLE
    return volume
print(volume_of_co2_at_stp(2.34))
```

In [27]:
def volume_of_co2_at_stp(mass):
    MOLAR_MASS_OF_CO2 = 44.01 # g/mol
    NUMBER_OF_MOLECULES_IN_A_MOLE = 6.02214076e23 # molecules/mol
    VOLUME_AT_STP = 22.4 # L/mol
    moles = mass / MOLAR_MASS_OF_CO2
    molecules = moles * NUMBER_OF_MOLECULES_IN_A_MOLE
    volume = molecules * VOLUME_AT_STP / NUMBER_OF_MOLECULES_IN_A_MOLE
    return volume
print(volume_of_co2_at_stp(2.34))

1.1910020449897751


##### chat
/prompt:chemistry
/nochain

Determine the number of moles of argon and the mass of argon in a sample of argon gas at STP that occupies 56.2 liters. 

##### response generated  by ChatGPT in 18 seconds at 1/7/2023 3:23:32 PM
```python
AVOGADRO_CONSTANT = 6.02214076e23  # atoms/mol
ARGON_MOLAR_MASS = 39.948  # g/mol
def moles_and_mass_of_argon(volume, molar_mass, avogadro_constant):
  moles = (volume * avogadro_constant) / molar_mass
  mass = moles * molar_mass
  return moles, mass
moles, mass = moles_and_mass_of_argon(56.2, ARGON_MOLAR_MASS, AVOGADRO_CONSTANT)
print(moles)
print(mass)
```

In [29]:
AVOGADRO_CONSTANT = 6.02214076e23  # atoms/mol
ARGON_MOLAR_MASS = 39.948  # g/mol
def moles_and_mass_of_argon(volume, molar_mass, avogadro_constant):
  moles = (volume * avogadro_constant) / molar_mass
  mass = moles * molar_mass
  return moles, mass
moles, mass = moles_and_mass_of_argon(56.2, ARGON_MOLAR_MASS, AVOGADRO_CONSTANT)
print(moles)
print(mass)

8.47212152578352e+23
3.3844431071200007e+25


##### chat

In your reply, only return Python code. Don't provide descriptions or comments. If you generate a function, make sure you write the call to that function as well and print out the result of calling that function to the console. Write a Python program that solves:

A 30.6 g sample of gas occupies 22.414 L at STP. What is the molecular weight of this gas?

##### response generated  by ChatGPT in 20 seconds at 1/4/2023 8:22:59 PM
```python
def volume_of_co2_at_stp(mass):
    MOLAR_MASS_OF_CO2 = 44.01 # g/mol
    NUMBER_OF_MOLECULES_IN_A_MOLE = 6.02214076e23 # molecules/mol
    AVOGADRO_CONSTANT = NUMBER_OF_MOLECULES_IN_A_MOLE # mol^-1
    VOLUME_OF_STANDARD_TEMPERATURE_AND_PRESSURE = 22.4 # liters
    moles_of_co2 = mass / MOLAR_MASS_OF_CO2
    return moles_of_co2 * VOLUME_OF_STANDARD_TEMPERATURE_AND_PRESSURE * AVOGADRO_CONSTANT
print(volume_of_co2_at_stp(2.34))
```

In [8]:
#%%capture output
def volume_of_co2_at_stp(mass):
    MOLAR_MASS_OF_CO2 = 44.01 # g/mol
    NUMBER_OF_MOLECULES_IN_A_MOLE = 6.02214076e23 # molecules/mol
    AVOGADRO_CONSTANT = NUMBER_OF_MOLECULES_IN_A_MOLE # mol^-1
    VOLUME_OF_STANDARD_TEMPERATURE_AND_PRESSURE = 22.4 # liters
    moles_of_co2 = mass / MOLAR_MASS_OF_CO2
    return moles_of_co2 * VOLUME_OF_STANDARD_TEMPERATURE_AND_PRESSURE * AVOGADRO_CONSTANT
print(volume_of_co2_at_stp(2.34))

7.172381960376277e+23
